In [1]:
import requests
import json
import datetime

In [31]:
USERNAME ='OOIAPI-9N9UMLHV9W5GOP'
TOKEN= 'SJN6HXHH116OZ8'

In [32]:
# define your inputs and starting time

ref_des = 'RS03CCAL-MJ03F-05-BOTPTA301'
stream = 'botpt_nano_sample'
delivery_method = 'streamed'
parameter = '7'
begin_time = datetime.date(2017, 01, 01)



In [33]:
# prepare your time stamps

begin_time_str = begin_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')
end_time = begin_time + datetime.timedelta(seconds=86400)
end_time_str = end_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')

ntp_epoch = datetime.datetime(1900, 1, 1)
unix_epoch = datetime.datetime(1970, 1, 1)
ntp_delta = (unix_epoch - ntp_epoch).total_seconds()

In [34]:
print begin_time_str, end_time_str

2017-01-01T00:00:00.000Z 2017-01-02T00:00:00.000Z


In [35]:
for i in range(10):
    response = requests.get('https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'+ref_des[:8]+'/'+ref_des[9:14]+'/'+ref_des[15:27]+'/'+delivery_method+'/'+stream+'?beginDT='+begin_time_str+'&endDT='+end_time_str+'&limit=1000&parameters='+parameter, auth=(USERNAME, TOKEN))
    begin_time = begin_time + datetime.timedelta(seconds=86400)
    begin_time_str = begin_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')
    end_time = end_time + datetime.timedelta(seconds=86400)
    end_time_str = end_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')
    data = response.json()
    timestamp = data[0]['time']
    timestamp = datetime.datetime.utcfromtimestamp(timestamp - ntp_delta).replace(microsecond=0)
    print timestamp

    


2017-01-01 00:00:00
2017-01-02 00:00:00
2017-01-03 00:00:00
2017-01-04 00:00:00
2017-01-05 00:00:00
2017-01-06 00:00:00
2017-01-07 00:00:00
2017-01-08 00:00:00
2017-01-09 00:00:00
2017-01-10 00:41:49


In [7]:
print json.dumps(response.json(), indent=2)

[
  {
    "pk": {
      "node": "MJ03F", 
      "stream": "botpt_nano_sample", 
      "subsite": "RS03CCAL", 
      "deployment": 1, 
      "time": 3692217600.0, 
      "sensor": "05-BOTPTA301", 
      "method": "streamed"
    }, 
    "time": 3692217600.0
  }, 
  {
    "pk": {
      "node": "MJ03F", 
      "stream": "botpt_nano_sample", 
      "subsite": "RS03CCAL", 
      "deployment": 1, 
      "time": 3692217686.35, 
      "sensor": "05-BOTPTA301", 
      "method": "streamed"
    }, 
    "time": 3692217686.35
  }, 
  {
    "pk": {
      "node": "MJ03F", 
      "stream": "botpt_nano_sample", 
      "subsite": "RS03CCAL", 
      "deployment": 1, 
      "time": 3692217772.85, 
      "sensor": "05-BOTPTA301", 
      "method": "streamed"
    }, 
    "time": 3692217772.85
  }, 
  {
    "pk": {
      "node": "MJ03F", 
      "stream": "botpt_nano_sample", 
      "subsite": "RS03CCAL", 
      "deployment": 1, 
      "time": 3692217859.35, 
      "sensor": "05-BOTPTA301", 
      "method": "st

In [8]:
data = response.json()
timestamp = data[0]['time']

<type 'float'>


In [43]:

newtime = timestamp / (24 * 3600) + datetime.datetime(1900, 1, 1).toordinal()
print newtime

736330.0


In [ ]:
def ntp_seconds_to_datetime(ntp_seconds):
    return datetime.datetime.utcfromtimestamp(ntp_seconds - ntp_delta).replace(microsecond=0)

last_time = data['time'][-1]
begin_time = ntp_seconds_to_datetime(last_time)
end_time = ntp_seconds_to_datetime(last_time) + datetime.timedelta(seconds=3600)

        
print json.dumps(response.json(), indent=2)

for i in range(10):
    print 'i'

for i in json_string:
    temp.append(i['seawater_temperature'])
    time_stamp = i['time']
    dtime = time_stamp / (24 * 3600) + datetime.datetime(1900, 1, 1).toordinal()
    time2.append(dtime)